In [1]:
import pandas as pd
import numpy as np
import sqlite3

pd.options.display.max_rows = 999
pd.options.display.max_columns = 100

# Precinct_d

In [26]:
# con = sqlite3.connect("db/votes.db")
# cursor = con.cursor()

# cd_map = pd.read_csv('data/delegate_apportionment_clean.csv', index_col=0)
# sqlite_insert_query = """INSERT INTO precinct_d
#                   (county, precinct, CD2SDE) 
#                   VALUES (?, ?, ?);"""
# recordList = cd_map.get_values()
# cursor.executemany(sqlite_insert_query, recordList)

# con.commit()
# con.close()

# Votes

In [6]:
cd_map = pd.read_csv('../data/delegate_apportionment_clean.csv', index_col=0)

In [8]:
cd_map.head()
cd.loc[:,'uuid'] = cd.loc['county'] + cd['precinct'] +

,county,precinct,sde
0,Lee,FM4B,0.000
1,Lee,KE2B,0.000
2,Lee,HR,0.000
3,Appanoose,CW,0.160
4,Black Hawk,S000,0.404


In [ ]:
pd.read_sql("")
np.random.choice(cd_map.index.values)

In [49]:
# Create fake precinct data
idx_choice = np.random.choice(cd_map.index.values)
test = cd_map.iloc[idx_choice:idx_choice + 1,:]
test.head()
del test['sde']
test.loc[:,'round'] = 2

var_list = ['buttigieg_votes', 'warren_votes', 'klobuchar_votes', 'biden_votes',
            'yang_votes', 'sanders_votes', 'steyer_votes', 'gabbard_votes', 
            'other_votes', 'uncommitted_votes']
for v in var_list:
    test.loc[:,v] = np.random.randint(0,100)
test.head()

,county,precinct,round,buttigieg_votes,warren_votes,klobuchar_votes,biden_votes,yang_votes,sanders_votes,steyer_votes,gabbard_votes,other_votes,uncommitted_votes
480,Linn,MR05,2.0,5.0,43.0,10.0,77.0,61.0,80.0,6.0,16.0,17.0,28.0


In [50]:
# results file
con = sqlite3.connect("db/votes.db")
cursor = con.cursor()

sqlite_insert_query = """INSERT INTO votes
                  (county, precinct, round, 
                  buttigieg_votes, warren_votes, klobuchar_votes, biden_votes, yang_votes, 
                  sanders_votes, steyer_votes, gabbard_votes, other_votes, uncommitted_votes) 
                  VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?);"""

recordList = test.get_values()
cursor.executemany(sqlite_insert_query, recordList)
con.commit()
con.close()

# Results

In [89]:
con = sqlite3.connect("db/votes.db")

# delete all previous results
cur = con.cursor()
cur.execute('DELETE FROM results')
con.commit()
con.close()

# insert new records
con = sqlite3.connect("db/votes.db")
cur = con.cursor()
df = pd.read_sql_query("SELECT * from votes WHERE round = 2", con)
df.head()

cd_map = pd.read_sql_query("SELECT * from precinct_d", con)
cd_map.head()

rdf = df.merge(cd_map, how='left', on=['county','precinct'])
vote_cols = [c for c in rdf.columns if '_votes' in c]
rdf.loc[:,'total_votes'] = rdf.loc[:, vote_cols].sum(axis=1)
for c in vote_cols:
    rdf.loc[:,c[:-6]+'_sdes'] = rdf.loc[:, c] / rdf['total_votes'] * rdf['CD2SDE']
    
del rdf['CD2SDE']
del rdf['round']
del rdf['total_votes']

sqlite_insert_query = """INSERT INTO results
                  (county, precinct, 
                  buttigieg_votes, warren_votes, klobuchar_votes, biden_votes, yang_votes, 
                  sanders_votes, steyer_votes, gabbard_votes, other_votes, uncommitted_votes,
                  buttigieg_sde, warren_sde, klobuchar_sde, biden_sde, yang_sde, 
                  sanders_sde, steyer_sde, gabbard_sde, other_sde, uncommitted_sde)
                  VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?,
                          ?, ?, ?, ?, ?, ?, ?, ?);"""

recordList = rdf.get_values()
cur.executemany(sqlite_insert_query, recordList)
con.commit()
con.close()

In [90]:
con = sqlite3.connect("db/votes.db")
df = pd.read_sql_query("SELECT * from results", con)
df.head()

,county,precinct,buttigieg_votes,warren_votes,klobuchar_votes,biden_votes,yang_votes,sanders_votes,steyer_votes,gabbard_votes,other_votes,uncommitted_votes,buttigieg_sde,warren_sde,klobuchar_sde,biden_sde,yang_sde,sanders_sde,steyer_sde,gabbard_sde,other_sde,uncommitted_sde
0,Shelby,7,65,98,93,10,88,35,48,57,88,19,0.054077,0.081531,0.077371,0.008319,0.073211,0.029118,0.039933,0.047421,0.073211,0.015807
1,Linn,MR05,5,43,10,77,61,80,6,16,17,28,0.051258,0.440816,0.102515,0.789369,0.625344,0.820123,0.061509,0.164025,0.174276,0.287043
